# Lab work journal

Notes for each sessions of lab work.

### 2018-04-10 Genome assembly
Made soft links to the data located in the /ga2018/data folders.
Started writing bash script for the batch assembling the genome, will have to wait for next lab session.
Want to test running megahit in an interactive session first to see that it works. 

### 2018-04-12 UPPMAX down
Uppmax down for maintenance. Working on preparation of the batch script so that it can be submitted as soon as UPPMAX comes online again.

### 2018-04-17 UPPMAX still down
Uppmax expected to be down until at least 4-20. Finalized the assembly batch script with the help of the lab assistants. Considering running the assembly on personal computer if the data becomes accessible.

Installed Quast locally.

Started working on BWA batch script, not much progress.

Downloaded data locally, will try to run megahit on my own.

### 2018-04-18 Trying locally
Trying to run the assembly using megahit locally. First try crashed after 3h due to not enough memory available. Freed up roughly 17GB and tried again, crashed again after 9h. Going to run on UPPMAX instead.

### 2018-04-19 Running assembly
Uppmax back up again. Submitted genome assembly as batch script assemble.sh at 13:30. Job failed due to output folder already existing (slurm output had error message). Resubmitted at 13:40.

### 2018-04-23 Completed assembly
First try at assembly failed due to time out after 8h. Resubmitted with 11h scheduled. This also timed out. After talking to lab assistants I defined which kmers should be used and resubmitted. Finally it worked after roughly 5,5h runtime. Yay.

### 2018-04-25 Mapping and quality checking
Started quast quality control of the assembly using metaquast.py locally without specifying reference genome. Sent batch job map_dna.sh. Crashed due to trying to run both samples (4 files) in the same run. Split into to jobs, one for SRR4342129 and one for SRR4342133. Submitted SRR4342129 first to see if it runs without crashing now. 
First job seemed to run fine, submitted second as well.

metaquast finished without problem in roughly 1h 20 mins.

Started MetaBat in working node 16:05. command: 
```
metabat -t 2 -i  /home/haol7814/ga2018/analyses/02_assembly/final.contigs.fa -o bin
```
MetaBat finished at 16:30 with 27 bins.

### 2018-05-09 Binning using coverage info
Got a tip to use the coverage information when binning since this can improve the result. MetaBat can take in the .bam (.sam in my case) files from bwa.
Renamed old folder for bins to OLD_05_binning (in case I want to compare later...)
Reran bwa and piped to samtools because .bam was needed instead of .sam.


### 2018-05-14 Actually binning and other things
Binning using command: 
```
metabat2 -t 2 -i /home/haol7814/ga2018/analyses/02_assembly/final.contigs.fa -a final.contigs.fa.depth.txt  -o bin
```
Resulted in 44 bins. 

Installed CheckM and added to bash profile in order for bash to be able to find the command. 
Made bash script quality_bins.sh and submitted. 
Ran command: 
```
checkm qa /home/haol7814/ga2018/analyses/06_qc_bins/lineage.ms /home/haol7814/ga2018/analyses/06_qc_bins
```
This resulted in a table with the completeness and contamination for the different bins. Saved to file bin_stats.txt.

Submitted batch script func_anno.sh for functional annotation using prokka.
Missed the option for taxonomic ids, resubmitted.

Installed phylophan locally on UPPMAX and submitted batch script phylo.sh.
Batch failed due to bin files having the .fa instead of .faa.
Changed using command:
```
for f in *.fa; do 
mv -- "$f" "${f%.fa}.faa"
done
```

Ran FastQC on raw RNA files and started looking into Trimmomatic.


#### 2018-5-16 Functional annotation issues
Both functional annotation (PROKKA) and the taxonomic placement (Phyophlan) exeeded time. Set prokka to --fast and retried but still not enough time (8h).

Changed to PROKKA to run on the bins (misunderstood and ran on the assembly). Divided into archaea and bacteria to  improve result 
(
command for making soft links for each bin: 
```
for g in ../../../05_binning/bins/bin.*; do ln -s $g; done
```
)

Bacteria is default but for archaea ```--Kingdom Archaea``` option was used.

Will run phylophlan on the output from PROKKA (bin.#.faa)

Looked further into the quality of the bins. Several bins have good completeness and the contamination also seems good. 

Discussed the quality of the assmebly. Hard for this to see quality from the quast report (since it tried to find reference genomes itself and there probably aren't that many simillar genomes in the databases. This environment is quite rare so the organisms here aren't well represented in databases.)
One indication of the quality of the assembly is how much coverage of the assembly has of the reads. Checked using command:
```samtools flagstat SRR....bam```
This showed roughly 54\% coverage. Not the best but this is probably due to only two samples being used.

Further steps in the RNA analysis can include mapping the RNA to the different bins and seeing how much maps to each bin/gene for identifying more or less expressed genes. 

Abundance of different organisms(bins) can be analyzed by mapping the reads to the bins and normalizing to the length of the genome(bin). The more mapped to the bin the more of that organism should be present. 

Soft linked files to phylophlan input using command:
```
for g in /home/haol7814/ga2018/analyses/07_func_anno/prokka/archaea/bin.*.fa_out/bin.*.faa; do ln -s $g; done
```

Submitted phylo.sh 

#### 2018-05-17 Last lab day

The plan is to map the RNA and DNA to the bins to find out about the the expression and the abundance of the different organisms. For this each bin should be treated by removing the headers so as to get a file with the raw sequences. In this analysis it doesn't matter on specifically which gene/chromosome, only the overall coverage stats. In the supplemental information from the paper they have formulas in the excel sheet Abund_calcs2.xlsx that can be used to get normalized values of the abundance and such. 
For finding out about what specific genes are expressed the annotation file (.gff) can be used with HtSeq.

Phylophlan crashed after 1h, this was due to not removing the project folder in the data folder in phylophlan. For each restart of phylophlan a new project name is needed or remove both the folder in /ouptput and in /data.

Started mapping DNA to the bins. Trying with one bin first (bin.10). Going to use samtools to analyze the bam files.

Also started mapping for the RNA expression analysis.
